# Simulations of effective dose on a surface


Effective dose is used to assess the potential for long-term effects that might occur in the future.

Effective dose provides a sinlge number that reflects the exposure to radiation. To quote ICRP who deffine the quantity "It sums up any number of different exposures into a single number that reflects, in a general way, the overall risk"

Effective dose is a calculated value, measured in milli Sv. Effective dose is calculated for the whole body. The sum of equivalent doses to all organs, each adjusted to account for the sensitivity of the organ to radiation. Read more about equivalent dose, absorbed dose and effective dose on the ICRP website.

http://icrpaedia.org/Absorbed,_Equivalent,_and_Effective_Dose

In [ ]:
The amount of effective dose deposited by a neutron or photon depends on the energy of the particle. The dose coeffients provided by ICRP are energy dependant.

The following section plots the 

In [1]:
energy_bins_n, dose_coeffs_n = openmc.data.dose_coefficients(particle='neutron', geometry='AP')
    energy_bins_p, dose_coeffs_p = openmc.data.dose_coefficients(particle='photon', geometry='AP')
    energy_function_filter_n = openmc.EnergyFunctionFilter(energy_bins_n, dose_coeffs_n)
    energy_function_filter_p = openmc.EnergyFunctionFilter(energy_bins_p, dose_coeffs_p)


IndentationError: unexpected indent (<ipython-input-1-379a1c1400a0>, line 2)